<a href="https://colab.research.google.com/github/SuyoungPark11/ml_learning_project/blob/main/DACON_bluehouse1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dacon.io/competitions/open/235597/overview/description

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# konlpy, Mecab 형태소 분석기 설치 스크립트 실행
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
import requests
from konlpy.tag import Okt
import numpy as np
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [ ]:
# import data
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv', encoding='utf-8').iloc[:, 1:]
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test.csv', encoding='utf-8').iloc[:, 1:]
sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sample_submission.csv', encoding = 'utf-8') 

In [ ]:
# category 0 : 인권/성평등, 1: 문화/예술/체육/언론, 2: 육아/교육
train.head()

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [ ]:
print(train.isnull().sum())
print("-------------------")
print(train.groupby('category').size())

index       0
category    0
data        8
dtype: int64
-------------------
category
0    13301
1    13337
2    13362
dtype: int64


In [ ]:
train = train.dropna()
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 품사 태깅
okt = Okt()
X_train = []
for sentence, i in zip(train['data'], tqdm(range(len(train['data'])))):
  temp_X = []
  # 형태소 단위 분석
  temp_X = okt.morphs(sentence, stem=True)
  temp_X = [word for word in temp_X if word not in stopwords]
  X_train.append(temp_X)
X_test = []
for sentence in test['data']:
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True)
  temp_X = [word for word in temp_X if word not in stopwords]
  X_test.append(temp_X)

# Updates internal vocabulary based on a list of texts. 
# In the case where texts contains lists, we assume each entry of the lists to be a token
# Using befroe texts_to_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = 30000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
# Transforms each text in texts to a sequence of integers.
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
# Transforms a list (of length num_samples) of sequences into a 2D Numpy array of shape
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
# Converts a class vector (integers) to binary class matrix.
y_train = to_categorical(train['category'])

100%|█████████▉| 39991/39992 [50:10<00:00, 13.29it/s]


In [ ]:
# Model
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 562s 2s/step - loss: 0.5346 - acc: 0.7882
Epoch 2/15
313/313 [==============================] - 606s 2s/step - loss: 0.2981 - acc: 0.8943
Epoch 3/15
313/313 [==============================] - 557s 2s/step - loss: 0.2434 - acc: 0.9131
Epoch 4/15
313/313 [==============================] - 558s 2s/step - loss: 0.1937 - acc: 0.9289
Epoch 5/15
313/313 [==============================] - 557s 2s/step - loss: 0.2205 - acc: 0.9225
Epoch 6/15
313/313 [==============================] - 556s 2s/step - loss: 0.1540 - acc: 0.9449
Epoch 7/15
313/313 [==============================] - 563s 2s/step - loss: 0.1320 - acc: 0.9534
Epoch 8/15
313/313 [==============================] - 640s 2s/step - loss: 0.1591 - acc: 0.9447
Epoch 9/15
313/313 [==============================] - 580s 2s/step - loss: 0.1196 - acc: 0.9574
Epoch 10/15
313/313 [==============================] - 557s 2s/step - loss: 0.1340 - acc: 0.9535
Epoch 11/15
313/313 [==================

In [ ]:
# Prediction
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/submission.csv', encoding='utf-8', index=False)